# Прогнозирование оттока клиентов провайдера

**Целью проекта** является выявление оттока клиентов из компании-оператора телефонной связи и интернета. Своевременное прогнозирование оттока позволит компании вовремя проводить удерживающие маркетинговые мероприятия и увеличивать лояльность среди клиентов, а так же сократить расходы на удержание таких клиентов.

Прогнозирование оттока клиентов будет решаться задачей классификацией с применением машинного обучения.

**Этапы проекта:**
1. **Обзор данных**:

    Загрузим данные в тетрадку и проведем их предварительный обзор для понимания дальнейших действий.
2. **Предобработка данных**:
    * объединим таблицы;
    * обработаем пропущенные значения;
    * обработаем аномалий и дубликаты;
    * приведем данные к нужным форматам.
3. **Исследовательский анализ данных**:
    * проверим данные на полноту и распределение;
    * проверим данные на аномалии;
    * проверим баланс классов целевой переменной;
    * проведем корреляционный анализ;
    * построим соответствующие графики.
4. **Подготовка данных для обучения:**
    * разобьем датасет на train и test выборки;
    * проведем кодирование категориальных фичей;
    * проведем масштабирование.
4. **Обучение моделей**:
    * обучим модели;
    * посмотрим на результат их работы;
    * для улучшения метрик проведем тюнинг гиперпараметров и накрутим кросс-валидацию;
    * для финальной оценки лучшей модели проверим модель на test выборке.
    
    Для оценки моделей будем использовать метрики AUR-ROC и Accuracy.
5. **Общий вывод**:

    Напишем общий вывод и отчет о проделанной работе.

## Обзор данных

Загрузим данные в тетрадку и проведем их предварительный обзор для понимания дальнейших действий.

In [1]:
# Импорт стандартных библиотек
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

# Импорт сторонних библиотек
import phik
from catboost.utils import get_roc_curve
from catboost import CatBoostClassifier, Pool

In [2]:
# Обработка исключений для подгрузки данных с севера или локального компьютера
try:
    contract = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\Data Science\\'
                           'Обучение в Яндекс\\17. Выпускной проект\\final_provider\\contract.csv')
    internet = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\Data Science\\'
                           'Обучение в Яндекс\\17. Выпускной проект\\final_provider\\internet.csv')
    personal = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\Data Science\\'
                           'Обучение в Яндекс\\17. Выпускной проект\\final_provider\\personal.csv')
    phone = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\Data Science\\'
                        'Обучение в Яндекс\\17. Выпускной проект\\final_provider\\phone.csv')
except:
    contract = pd.read_csv('/datasets/final_provider/contract.csv')
    internet = pd.read_csv('/datasets/final_provider/internet.csv')
    personal = pd.read_csv('/datasets/final_provider/personal.csv')
    phone = pd.read_csv('/datasets/final_provider/phone.csv')

Выедем датасеты и информацию о них на экран.

In [3]:
display(contract.info())
display(contract)
display(contract.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


None

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6


,count,mean,std,min,25%,50%,75%,max
MonthlyCharges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


Датасет состоит из 8 колонок и 7043 строк, описывающих контрактную часть с клиентами:
* customerID - ID покупателя;
* BeginDate - дата начала действия договора;
* EndDate - дата окончания договора;
* Type - тип договора по сроку действия;
* PaperlessBilling - безбумажное заполнение договора;
* PaymentMethod - метод оплаты договора;
* MonthlyCharges - ежемесячные платежи;
* TotalCharges - итоговая сумма платежей.

Средняя сумма ежемесячных платежей среди клиентв составляет 64.76 ye., медиана 70.35 ye.

In [4]:
display(internet.info())
display(internet)
display(internet.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


None

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...
5512,6840-RESVB,DSL,Yes,No,Yes,Yes,Yes,Yes
5513,2234-XADUH,Fiber optic,No,Yes,Yes,No,Yes,Yes
5514,4801-JZAZL,DSL,Yes,No,No,No,No,No
5515,8361-LTMKD,Fiber optic,No,No,No,No,No,No


,count,unique,top,freq
customerID,5517,5517,7590-VHVEG,1
InternetService,5517,2,Fiber optic,3096
OnlineSecurity,5517,2,No,3498
OnlineBackup,5517,2,No,3088
DeviceProtection,5517,2,No,3095
TechSupport,5517,2,No,3473
StreamingTV,5517,2,No,2810
StreamingMovies,5517,2,No,2785


Датасет состоит из 8 колонок и 5517 строк, описывающих интернет услуги:

* InternetService - тип интернет соединения;
* OnlineSecurity - онлайн безопасность;
* OnlineBackup - резервная копия данных;
* DeviceProtection - антивирус;
* TechSupport - поддержка;
* StreamingTV - стриминговое телевидение;
* StreamingMovies - каталог фильмов.

Соотношение покупаемых и непокупаемых услуг в пользу непокупаемых. Каталог фильмов берут чаще, чем остальные услуги. Преобладающий тип интернет соединения - оптоволокно.

In [5]:
display(personal.info())
display(personal)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


None

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No
...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes
7039,2234-XADUH,Female,0,Yes,Yes
7040,4801-JZAZL,Female,0,Yes,Yes
7041,8361-LTMKD,Male,1,Yes,No


Датасет состоит из 5 колонок и 7043 строк, описывающих клиентов:

* InternetService - тип интернет соединения;
* gender - пол;
* SeniorCitizen - пожилой возраст;
* Partner - партнер компании;
* Dependents - инживенец.

In [6]:
display(phone.info())
display(phone)
display(phone.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


None

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes
...,...,...
6356,2569-WGERO,No
6357,6840-RESVB,Yes
6358,2234-XADUH,Yes
6359,8361-LTMKD,Yes


,count,unique,top,freq
customerID,6361,6361,5575-GNVDE,1
MultipleLines,6361,2,No,3390


Датасет состоит из 2 колонок и 6361 строк, описывающих клиентов с подключенной телефонией:

* InternetService - тип интернет соединения;
* MultipleLines - множественные линии.
 
 Среди пользователей преобладает телефония с 1 линией.

Выведем на экран все уникальные уникальные значения и их встречаемость в датасетах с целью посмотреть на скрытые аномалии.

In [7]:
# Вывод уникальных значений каждого признака, отсортированных по убыванию
datasets = {'contract': contract,
            'internet': internet,
            'personal': personal,
            'phone': phone}

for key, value in datasets.items():
    display(key)
    for column in value:
        display(column)
        display(value[column].value_counts().sort_values())
        display()
    display('-----------------------------------------------------------------------------------------')

'contract'

'customerID'

7590-VHVEG    1
9237-HQITU    1
9305-CDSKC    1
1452-KIOVK    1
6713-OKOMC    1
             ..
8361-LTMKD    1
1122-JWTJW    1
4807-IZYOZ    1
6894-LFHLY    1
3186-AJIEK    1
Name: customerID, Length: 7043, dtype: int64

'BeginDate'

2013-11-01      2
2013-12-01      3
2013-10-01      3
2014-01-01      7
2020-02-01     11
             ... 
2020-01-01    233
2019-09-01    237
2019-11-01    237
2019-10-01    237
2014-02-01    366
Name: BeginDate, Length: 77, dtype: int64

...

'customerID'

5575-GNVDE    1
7892-POOKP    1
6388-TABGU    1
9763-GRSKD    1
7469-LKBCI    1
             ..
4807-IZYOZ    1
5306-BVTKJ    1
1471-GIQKQ    1
0376-OIWME    1
3186-AJIEK    1
Name: customerID, Length: 6361, dtype: int64

'MultipleLines'

Yes    2971
No     3390
Name: MultipleLines, dtype: int64

'-----------------------------------------------------------------------------------------'

### Вывод

Из проведенного обзора данных можно сделать следующие выводы:
* колонки BeginDate и EndDate необходимо привести к формату DateTime в целях исследовательского анализа данных. После анализа колонки можно будет удалить, т.к. они будут лишними для моделей;
* колонка EndDate является целевым признаком для классификации клиентов на активных и прогнозируемых к оттоку. Колонка имеет смешанный формат данных. Необходимо привести ее к бинарному виду для дальнейшей классификации клиентов на активных (0) и прогнозируемых к оттоку (1). Но, в связи с тем, что сама по себе информация о окончании действия договора, возможно, несет в себе практическую пользу для моделей (можно сгенерировать отдельную фичу с продолжительность действия договора и посмотреть как она влияет на метрики моделей), перед приведением целевого признака в бинарный вид мы сгенерируем дополнительную фичу с помощью колонок BeginDate и EndDate;
* колонку TotalCharges необходимо привести к float формату, дополнительно в колонке есть скрытые пропуски из-за чего, возможно, формат колонки типа object. Заменим их на 0 или удалим.
* необходимо провести кодирование категориальных колонок в каждом из датасетов;
* необходимо произвести масштабирование категориальных колонок;
* полезность фичи MonthlyCharges для моделей под вопросом, т.к. платежи гомогенны, без динамики. Можно попробовать посмотреть метрики с этой фичей и без;
* данные необходимо будет объединить в один датафрейм.

## Предобработка данных

Объединим таблицы, чтобы проще было работать с данными.

In [8]:
# Объединение датасетов в один фрейм
data = contract.merge(internet, how='left', on='customerID')
data = data.merge(personal, how='left', on='customerID')
data = data.merge(phone, how='left', on='customerID')
data

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5,DSL,Yes,No,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9,Fiber optic,No,Yes,Yes,No,Yes,Yes,Female,0,Yes,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,NaN
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes


### Проверка на пропуски

Проверим датасет на пропуски.

In [9]:
data.isna().sum()

customerID             0
BeginDate              0
EndDate                0
Type                   0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
InternetService     1526
OnlineSecurity      1526
OnlineBackup        1526
DeviceProtection    1526
TechSupport         1526
StreamingTV         1526
StreamingMovies     1526
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
MultipleLines        682
dtype: int64

Видно, что в данных присутствуют пропуски из-за несоответствия размерности таблиц Contract, Internet и Phone. Посмотрим поконкретнее, что это за пропуски. Допустим, имеются ли такие клиенты, которые не пользуются ни интернетом ни телефонией?

In [10]:
# Проверка датасета на NAN
data.loc[(data['InternetService'].isnull()) & (data['MultipleLines'].isnull())]

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines


Клиентов, которые не пользуются ни интернетом ни телефонией нет, соответсвенно все клиенты у нас задействованы в обслуживании и всех их можно брать в расчет модели (не удалять пропуски).

В связи с тем, что в столбцах имеются пропуски - заменем их соответствующими значениями. Т.к. пропуски во всех категориальных столбцах с одинаковым бинарным разбиением (Yes or No) - заменим все NAN значения на "No", как не употребляемые клиентами услуги.

In [11]:
# Замена NAN на 'No'
columns = []

# Цикл, ищущий NAN в кололнках
for column in data.columns:
    if data[column].isnull().sum() > 0:
        columns.append(column)

display(f'Колонки с NAN: {columns}')

# Цикл, заменяющий найденные в колонках NAN на 'No'
for column in columns:
    data[column] = data[column].fillna('No')

display(data.isnull().sum())
display(data)

"Колонки с NAN: ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']"

customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
MultipleLines       0
dtype: int64

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,No
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,No
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5,DSL,Yes,No,Yes,Yes,Yes,Yes,Male,0,Yes,Yes,Yes
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9,Fiber optic,No,Yes,Yes,No,Yes,Yes,Female,0,Yes,Yes,Yes
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,No
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes


### Проверка на дубликаты

Проверим датасет на явные дубликаты.

In [12]:
data.duplicated().sum()

0

Явные дубликаты в данных отсутствуют.

### Удаление неявных пропусков и приведение типов

При обозоре данных мы выявили, что в колонке TotalCharges имеются аномалии в виде пустой строки. Т.к. данные в других столбцах могут быть полезными - не будем удалять 11 аномалий, а заменим их на нули.

In [13]:
data['TotalCharges'].value_counts()

          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6531, dtype: int64

data['TotalCharges'] = data['TotalCharges'].replace([' '], 0)
data['TotalCharges'] = data['TotalCharges'].replace([' '], data['TotalCharges'].median())
data['TotalCharges'].value_counts()

In [14]:
data = data[data['TotalCharges'] != ' ']

In [15]:
data['TotalCharges'].value_counts()

20.2      11
19.75      9
20.05      8
19.9       8
19.65      8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6530, dtype: int64

In [16]:
data.loc[data['TotalCharges'] == 0]

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines


Теперь приведем колонку TotalCharges к типу float.

In [17]:
data['TotalCharges'] = data['TotalCharges'].astype(float)

C:\Users\BLACKE~1\AppData\Local\Temp/ipykernel_8276/1864821250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TotalCharges'] = data['TotalCharges'].astype(float)


In [18]:
data['TotalCharges'].dtypes()

TypeError: 'numpy.dtype[float64]' object is not callable

Все колонки приведены к необходимым форматам.

### Майнинг таргета и дополнительных фич

Выделем таргет в отдельный столбец, чтобы произвести манипуляции со столбцами BeginDate и EndDate, вычислив CLF (customerlifecycle). Также приведем таргет в бинарный вид.

In [ ]:
data['target'] = data['EndDate']
data.head()

In [ ]:
'''Цикл для преобразования уникальных значений
в признаке target в бинарную классификацию (1 или 0).
Тем самым мы майним таргет.
'''


for i in data['target'].unique():
    if i != 'No':
        data['target'] = data['target'].replace(i, 1)
    data['target'] = data['target'].replace(i, 0)

data.head()

In [ ]:
'''Значения "No" в EndDate заменим на дату "2020-02-01"
в целью дальнейшего расчета customer life cycle.
Также проведем необходимые преобразования типов.
'''


data['EndDate'] = data['EndDate'].replace('No', '2020-02-01')
data['BeginDate'] = data['BeginDate'].astype('datetime64')
data['EndDate'] = data['EndDate'].astype('datetime64')

data[['BeginDate', 'EndDate']].head()

In [ ]:
data['customer_life_cycle'] = ((data['EndDate'] - data['BeginDate']).astype('timedelta64[D]')
                                                                    .astype('int'))
data.head()

### Вывод

В рамках первичной предобработки данных мы выполнили проверку датасета на пропуски, дубликаты, аномалии. Дополнительно нами был замайнена новая фича и определена целевая переменная. Первичная предобработка данных позволит нам провести ислледовательский анализ данных без артефактов в построенных графиках.

Вторичную обработку данных, а вернее их подготовку к машинному обучению будем делать в пункте 1.4 проекта.

## Исследовательский анализ данных

Проведем исследовательский анализ данных:
* проверим данные на полноту и распределение;
* проверим данные на аномалии и дубликаты (возможо они проявятся на графиках);
* проверим баланс классов;
* построим соответствующие графики.

### Проверка на полноту, распределение и аномалии

Посмотрим на распределение данных и отсутствие в их аномалий.

In [ ]:
plt.figure(figsize=(15, 10))

plt.hist(data['BeginDate'], bins=20)
plt.xticks(rotation=90)
plt.xlabel('Дата')
plt.ylabel('Количество заключенных договоров')

Дата начала договоров равномерно распределена по годам за исключением 2014 года и конца 2019 года. Возможно в 2014 году компания начала деятельность и заключила своим первые договоры. Причина заключения большого количества договоров в 2019 году неизвестна и требует отдельного анализа.

In [ ]:
plt.figure(figsize=(15, 10))

plt.hist(data['EndDate'], bins=20)
plt.xticks(rotation=90)
plt.xlabel('Дата')
plt.ylabel('Оток клиентов')

Отток клиентов компании равномерный. Фиксировать отток компания, видимо, начала в 2019 и делает это ежемесячно в 1 числах месяца. Большой отток 2020-02-01 не аномалия, а следствие нашей предобработки данных. Эта информация не несет пользы для нас.

Посмотрим на распределение ежемесячных платежей среди ушедших пользователей по сравнению с клиентами, которые до сих пор пользуются услугами компании.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plt.hist(data.loc[data['target'] == 0]['MonthlyCharges'])
plt.hist(data.loc[data['target'] == 1]['MonthlyCharges'])
plt.xlabel('Сумма ежемесячных платежей')
plt.ylabel('Количество заключенных договоров')

Распределения по ежемесячным платежам одинаковы. Также видна мультимодальность распределения (несколько пиков).

Посмотрим на распределение итоговых сумм платежей среди ушедших пользователей по сравнению с клиентами, которые до сих пор пользуются услугами компании.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plt.hist(data.loc[data['target'] == 0]['TotalCharges'])
plt.hist(data.loc[data['target'] == 1]['TotalCharges'])
plt.xlabel('Сумма итоговых платежей')
plt.ylabel('Количество заключенных договоров')

Распределение итоговых платежей тоже одиковое.

Теперь посмотрим на распределение жизненного цикла клиента.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
ax.xaxis.set_major_locator(ticker.MultipleLocator(100))

plt.hist(data.loc[data['target'] == 0]['customer_life_cycle'], bins=20)
plt.hist(data.loc[data['target'] == 1]['customer_life_cycle'], bins=20)
plt.xlabel('Customer life cycle')
plt.ylabel('Количество заключенных договоров')

Ушедшие клиенты чаще всего пользовались услугами компании короткое время (около 100 дней). В целом утечка клиентов есть, но она не привязана к каким-то событиям.

### Баланс классов целевой переменной

Проверим балансировку классов у целевого признака.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

plt.hist(data['target'])
plt.xlabel('Целевой признак')
plt.ylabel('Количество заключенных договоров')

Из гистограммы видно, что ушедших пользователей в несколько раз меньше, чем активных. В связи с этим при обучении моделей машинного обучения будем использовать балансировку классов.

### Анализ выбросов

Посмотрим на выбросы в ежемесячных платежах и общей сумме платежей.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 2))
plt.boxplot(data['MonthlyCharges'], vert=False)
ax.set_title('Диаграмма размаха значений итоговой цены')
ax.set_xlim(0, 200)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 2))
plt.boxplot(data['TotalCharges'], vert=False)
ax.set_title('Диаграмма размаха значений итоговой цены')
ax.set_xlim(0, 15000)

Выбросов не выявлено. Все значения находятся в диапазоне полуторного межквартильного расстояния.

### Корреляционный анализ

Воспользуемся библиотекой phik для оценки корреляции различных гиперпараметров между собой.

In [ ]:
corr_matrix = data.phik_matrix()
corr_matrix

Выведем коррелционные данные на тепловую карту для удобства анализа.

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(corr_matrix, annot=True)

Исходя из тепловой карты видно, что у нас высокая корреляция между показателем EndDate и target, что логично, учитывая, что эти показатели идентичны. Аналогичная ситуация с BeginDate и customer_life_cycle.

Также высокая корреляция наблюдается между показателями customer_life_cycle и TotalCharges, MontlyCharges и InternetService.

Признак gender позал отсутствие корреляции почти со всеми показателями. Скорее всего данный признак не даст какого-либо прироста метрики моделей машинного обучения. Мы можем его удалить.

Таже удалим мультиколлинеарные признаки BeginDate и EndDate.

### Вывод

Мы провели краткий исследовательский анализ данных. Нами были оценены распределение, и полнота данных, определена необходимость балансироваки классов в рамках машинного обучения, произведен анализ выбросов количественных признаков с помощью ящиков с усами, проведен корреляционный анализ.

В рамках полноты и распределение данных - порядок, - в данных нет никаких пропусков, которые мы могли не заметить в рамках первичной предобработки данных.

Модели машинного обучения требуют балансировки классов, т.к. разница между утекшими и активными пользователями существенна.

Исходя из ящиков с усами выбросов количественных данных у нас нет, их нормализация не трубуется.

Корреляционный анализ показал зависимость таких признаков, как:
* EndDate и target;
* BeginDate и customer_life_cycle;
* customer_life_cycle и TotalCharges;
* MontlyCharges и InternetService.

Признак gender не коррелирует ни с одним из других фичей, в связи с чем мы можем удалить его из датасета на этапе подготовки данных для обучения.

## Подготовка данных для обучения

### Удаление лишних признаков

Удалим признаки, которые помешают модели обучиться.

In [ ]:
data = data.drop(columns=[
    'customerID',
    'BeginDate',
    'EndDate',
    'gender',
    'TotalCharges'], axis=1)
data.columns

### Разделение датасета на обучающую и тестовую выборки

Разделим датасет на фичи и таргет.

In [ ]:
features = data.drop('target', axis=1)
target = data['target']

display(features.shape)
display(target.shape)
display(features.head())

Разделим датасет на обучающую выборку, тестовую выборку и соответствующие таргеты.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=.25,
                                                    random_state=160123)

display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

### Кодирование категориальных переменных

Применим кодирование категориальных переменных с помощью OneHotEncoding.

In [ ]:
# Сброс индексов для избежания появления NAN при join датасетов
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

ohe_columns = []

# Сбор категориальных признаков через цикл в список
for column in X_train.columns:
    if X_train[column].dtypes == object:
        ohe_columns.append(column)

# Обучение энкодера
encoder = OneHotEncoder(drop='first').fit(X_train[ohe_columns])
new_columns = encoder.get_feature_names(ohe_columns)

# Кодирование категориальных признаков, сбор train выборки
encoder_train = pd.DataFrame(encoder.transform(X_train[ohe_columns]).toarray(),
                             columns=new_columns)

X_train = X_train.join(encoder_train)
X_train = X_train.drop(columns=ohe_columns, axis=1)
display(X_train)

# Кодирование категориальных признаков, сбор test выборки
encoder_test = pd.DataFrame(encoder.transform(X_test[ohe_columns]).toarray(),
                            columns=new_columns)

X_test = X_test.join(encoder_test)
X_test = X_test.drop(columns=ohe_columns, axis=1)
display(X_test)

### Вывод

В данном разделе нами были удалены признаки, не несущие пользы для машинного обучения (ID пользователей, даты).

## Обучение моделей

Обучим несколько моделей на данных и посмотрим на метрику AUC-ROC.

Будем собирать pipelin'ы для каждой модели. В нем будет scaler и модель. Сетку параметров укажем в словаре param_grid. Обучим модели черех GridSearchCV с помощью кросс-валидации с 5 фолдами.

Для обучения будем использовать следующие модели:
* LogisticRegression
* RandomForestClassifier
* CatBoostClassifier

### LogisticRegression

In [ ]:
pipeline = Pipeline([("scaler", StandardScaler()),
                     ("lr", LogisticRegression(random_state=160123,
                                               class_weight='balanced'))])

param_grid = {
    'lr__solver': ['lbfgs', 'liblinear']
}

grid_pipeline = GridSearchCV(pipeline,
                             param_grid,
                             verbose=1,
                             cv=5,
                             scoring='roc_auc')

grid_pipeline.fit(X_train, y_train)
display(grid_pipeline.best_params_)
display(grid_pipeline.best_score_)

### RandomForestClassifier

In [ ]:
pipeline = Pipeline([("scaler", StandardScaler()),
                     ("rfc", RandomForestClassifier(random_state=160123,
                                                    class_weight='balanced'))])

param_grid = {
    'rfc__max_depth': [1, 10, 50],
    'rfc__n_estimators': [10, 50, 100]
}

grid_pipeline = GridSearchCV(pipeline,
                             param_grid,
                             verbose=1,
                             cv=5,
                             scoring='roc_auc')

grid_pipeline.fit(X_train, y_train)
display(grid_pipeline.best_params_)
display(grid_pipeline.best_score_)

### CatboostClassifier

In [ ]:
pipeline = Pipeline([("cbc", CatBoostClassifier(random_state=160123,
                                                auto_class_weights='Balanced'))])

param_grid = {
    'cbc__learning_rate': [0.01, 0.1],
    'cbc__depth': [1, 10]
}

grid_pipeline = GridSearchCV(pipeline,
                             param_grid,
                             verbose=1,
                             cv=5,
                             scoring='roc_auc')

grid_pipeline.fit(X_train, y_train)
display(grid_pipeline.best_params_)
display(grid_pipeline.best_score_)

### Проверка лучшей модели на тестовой выборке

In [ ]:
model = CatBoostClassifier(random_state=160123,
                           auto_class_weights='Balanced',
                           depth=10,
                           learning_rate=0.1)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
predictions_proba = model.predict_proba(X_test)[:, 1]

display(roc_auc_score(y_test, predictions_proba))
display(accuracy_score(y_test, predictions))

In [ ]:
catboost_pool = Pool(X_test, y_test)
(fpr, tpr, thresholds) = get_roc_curve(model, catboost_pool, plot=True)

In [ ]:
features_imp = pd.DataFrame(model.get_feature_importance(), X_test.columns)
features_imp

### Вывод

Из обучения и проверки моделей видно, что лучше всего себя показала модель CatBoostClassifier. AUC-ROC данной модели составляет 88% на обучающей выборке и 89.38% на тестовой. Accuracy модели на тестовой выборке состваляет 83.67%.

Важнейшими параметрами для обучения модели оказались MonthlyCharges и customer_life_cycle, отражающие ежемесячные платежи и время жизни клиента.

# Отчет о проделанной работе

В данном работе мы исследуем проблему оттока пользователей интернет/телеком-провайдера «Ниединогоразрыва.ком». Конечной целью нашего исследования будет построение прогнозной модели, основывающейся на машинном обучении, определяющей клиентов склонных к оттоку из компании. Результатом использования такой модели будет удержание клиентов внутри компании различными маркетинговыми схемами.
Анализируя имеющиеся данные по клиентам компании модель определяет к какой категории относится клиент:
* "0" - не склонен к оттоку;
* "1" - склонен к оттоку.

Для оценки качества модели нами используется метрика оценки бинарной классификации AUC-ROC (Area Under Curve - Receiver Operating Characteristic).
Обученная в работе модель позволяет определять склонность клиента к оттоку с точностью до 89%.

## Введение

Различные компании, ведя свою деятельность, часто не задумываются о необходимости построения баз данных для накопления статистических данных по своим различным бизнес-процессам. Но те компании, которые занялись этим вопросом заранее, по прошествию определенного количества времени получили преимущество по сравнению с конкуретами - это преимущество заключено в силе данных.

Одной из таких компаний является компания «Ниединогоразрыва.ком». Компания занималась сбором данных по своим клиентам и накопила внушительную базу данных, данные из которой можно использовать для создания различных управленческих рычагов.

Проанализировав базу можно понять, что у нас есть все необходимые данные для получения возможности в сохранении компанией своих клиентов. Эти возможности открываются нам за счет машинного обучения и статистики.

**Для обучения моделей проведем следующие этапы проекта:**
1. **Обзор данных**:

    Загрузим данные в тетрадку и проведем их предварительный обзор для понимания дальнейших действий.
2. **Предобработка данных**:
    * объединим таблицы;
    * обработаем пропущенные значения;
    * обработаем аномалий и дубликаты;
    * приведем данные к нужным форматам.
3. **Исследовательский анализ данных**:
    * проверим данные на полноту и распределение;
    * проверим данные на аномалии;
    * проверим баланс классов целевой переменной;
    * проведем корреляционный анализ;
    * построим соответствующие графики.
4. **Подготовка данных для обучения:**
    * разобьем датасет на train, valid, test выборки;
    * проведем кодирование категориальных фичей;
    * проведем масштабирование.
4. **Обучение моделей**:
    * обучим парочку "голых" моделей без гиперпараметров (LogisticRegression, RandomForest, Catboost и т.д.). Посмотрим на результат их работы;
    * для улучшения метрик проведем тюнинг гиперпараметров и накрутим кросс-валидацию;
    * для финальной оценки лучшей модели проверим модель на test выборке.
    
    Для оценки моделей будем использовать метрики AUR-ROC и accuracy.
5. **Общий вывод**:

    Напишем общий вывод и отчет о проделанной работе.

## Данные

Данные были взяты из базы данных компании «Ниединогоразрыва.ком». Данные разбиты на несколько датасетов:
   * Список контрактов.
   * Список клиентов.
   * Покупаемые интернет услуги.
   * Покупаемые услуги телефонии.

Проведя первичный обзор датасетов можно выделить следующие моменты:
1. У нас имеются следующие данные, они же признаки для дальнейшего обучения нашей модели:
    * `customerID` - ID покупателя;
    * `BeginDate` - дата начала действия договора;
    * `EndDate` - дата окончания договора;
    * `Type` - тип договора по сроку действия;
    * `PaperlessBilling` - безбумажное заполнение договора;
    * `PaymentMethod` - метод оплаты договора;
    * `MonthlyCharges` - ежемесячные платежи;
    * `TotalCharges` - итоговая сумма платежей;
    * `InternetService` - тип интернет соединения;
    * `OnlineSecurity` - онлайн безопасность;
    * `OnlineBackup` - резервная копия данных;
    * `DeviceProtection` - антивирус;
    * `TechSupport` - поддержка;
    * `StreamingTV` - стриминговое телевидение;
    * `StreamingMovies` - каталог фильмов.
    * `InternetService` - тип интернет соединения;
    * `gender` - пол;
    * `SeniorCitizen` - пожилой возраст;
    * `Partner` - партнер компании;
    * `Dependents` - инживенец;
    * `InternetService` - тип интернет соединения;
    * `MultipleLines` - множественные линии.
2. Всего у нас имеются 7043 строк с данными, соответственно столько же клиентов и заключенных контрактов.
3. В данных есть множественные недочеты, которые необходимо будет исправить:
    * колонки `BeginDate` и `EndDate` необходимо привести к формату DateTime в целях исследовательского анализа данных. После анализа колонки можно будет удалить, т.к. они будут лишними для моделей;
    * колонка `EndDate` является целевым признаком для классификации клиентов на активных и прогнозируемых к оттоку. Колонка имеет смешанный формат данных. Необходимо привести ее к бинарному виду для дальнейшей классификации клиентов на активных (0) и прогнозируемых к оттоку (1). Но, в связи с тем, что сама по себе информация о окончании действия договора, возможно, несет в себе практическую пользу для моделей (можно сгенерировать отдельную фичу с продолжительность действия договора и посмотреть как она влияет на метрики моделей), перед приведением целевого признака в бинарный вид мы сгенерируем дополнительную фичу с помощью колонок `BeginDate` и `EndDate`;
    * колонку `TotalCharges` необходимо привести к float формату, дополнительно в колонке есть скрытые пропуски из-за чего, возможно, формат колонки типа object. Заменим их на 0 или удалим.
    * необходимо провести кодирование категориальных колонок в каждом из датасетов;
    * необходимо произвести масштабирование данных;
    * полезность признаков `MonthlyCharges` и `TotalCharges` для моделей под вопросом, т.к. платежи гомогенны, без динамики. Можно попробовать посмотреть метрики с этой фичей и без;
    * данные необходимо будет объединить в один датафрейм.
    * данные необходимо разбить на train и test выборки.
    
Всего данные прошли 2 стадии обработки:
1. Предварительная обработка для исследовательского анализа:
    * проверка и удаление пропусков;
    * проверка и удаление дубликато;
    * проверка на неявные аномалии;
    * майнинг таргета и дополнительных признаков.
2. Вторичная обработка для пригодности к машинному обучению
    * удаление лишних для модели признаков, дающих утечку;
    * разделение датасета на train и test выборки в соотношении 75/25 (5274 против 1758 строк). Параметр random_state для разделения данных выбран 160123.
    * кодирование категориальных переменных с помощью GridSearchCV.

## Модели

Всего для выполнения работы нами были выбраны 3 модели:
* Logistic Regression.
* RandomForest Classifier.
* CatBoost Classifier.

Для каждой модели был составлен Pipeline с помощью которого данные для моделей прошли следующие стадии:
1. Предобработка:
    * масштабирование числовых признаков.
2. Составление обучающей сетки гиперпараметров.
3. Обучение каждой модели на данных с применением кросс-валидации с итерацией равной 5.

Для моделей были выбраны следующие сетки гиперпараметров:

**LogisticRegression:**
* 'lr__solver': ['lbfgs', 'liblinear']


**RandomForestClassifier:**
* 'rfc__max_depth': [1, 10, 50],
* 'rfc__n_estimators': [10, 50, 100]

**CatBoostClassifier:**
* 'cbc__learning_rate': [0.01, 0.1],
* 'cbc__depth': [1, 10]

Гиперпараметр *random_state* для всех моделей использовался равный 16012023, а классы были сбалансированы методом class_weight='balanced'.

Список признаков, которые были использованы для обучения моделей (в т.ч. закодиронные признаки):
1. Дефолтные:
    * MonthlyCharges'
    * SeniorCitizen'
    * customer_life_cycle'
2. Закодированные:
    * Type_One year'
    * Type_Two year'
    * PaperlessBilling_Yes'
    * PaymentMethod_Credit card (automatic)'
    * PaymentMethod_Electronic check'
    * PaymentMethod_Mailed check'
    * InternetService_Fiber optic'
    * InternetService_No'
    * OnlineSecurity_Yes'
    * OnlineBackup_Yes'
    * DeviceProtection_Yes'
    * TechSupport_Yes'
    * StreamingTV_Yes'
    * StreamingMovies_Yes'
    * Partner_Yes'
    * Dependents_Yes'
    * MultipleLines_Yes'

## Результаты обучения моделей

### Общая информация

Модели были успешно обучены на train выборке и показали следующие результаты:

**LogisticRegression:**

Лучшие гиперпараметры:
* 'lr__solver': 'lbfgs'

Лучшая целевая метрика (AUC-ROC):
`0.8386`

\
**RandomForestClassifier:**

Лучшие гиперпараметры:
* 'rfc__max_depth': 10
* 'rfc__n_estimators': 100

Лучшая целевая метрика (AUC-ROC):
`0.8523`

\
**CatBoostClassifier:**

Лучшие гиперпараметры:
* 'cbc__depth': 10
* 'cbc__learning_rate': 0.1

Лучшая целевая метрика (AUC-ROC):
`0.8801`

### Лучшая модель

**По итогам обучения моделей как лучшая из них была выбрана модель CatBoost Classifier с итоговой метрикой AUC-ROC равной `88.01%`.**

**Лучшие гиперпараметры:**
* 'cbc__depth': 10
* 'cbc__learning_rate': 0.1

**Лучшая целевая метрика (AUC-ROC) на train:**
`0.8801`

\
**Модель CatBoostClassifier, как лучшая, была проверена на тестовой выборке и получила на ней слудующие результаты:**

* AUC-ROC: `0.8938`

* Accuracy: `0.8367`

## Выводы

Нами выполнена работа по обучению моделей машинного обучения на данных компании «Ниединогоразрыва.ком».

Работа потребовала от нас предварительной обработки данных и их анализа. Так, например, были удалены некоторые пропуски в данных, замайнен из начальной и конечной даты контракта целевой признак (target) и дополнительный признак (customer_life_cycle).

Категорильные признаки в работе были закодированы, а все числовые признаки отмасштабированы.

К моделям машинного обучения был применеН PipeLine, перекрестная валидация с 5 фолдами и поиск по сетке гиперпараметров.

лучшей моделью по итогам обучения на обучающей выборке была определена модель CatBoostClassifier с AUC-ROC равной 88.01%. Данная модель была взята за основную и проверена на тестовой выборке, где показала еще лучший результат, равный 89.38%.

Данную модель можно брать за основу и применить к текущим данным компании, для определения клиентов, которые, скорее всего, являются утекающими. Таким клиентам можно предлагать скидки и дополнительные услуги, которые могут быть так же определены в рамках составленного с помощью машинного обучения портрета целевой аудитории (персонализированные услуги).

Обученную модель также можно применять к новым клиентам в будущем для исключения их утечки.

**По итогам работы отклонения от первоначального плана работы не было. Трудности в работе не выявлены. Ключевым признаком для обучения модели оказался customer_life_cycle.**